# Content
The dataset contains credit card transactions made by Brazilian cardholders. This dataset presents transactions over a two-day period, including 492 fraudulent cases out of a total of 284,807 transactions. The dataset is highly imbalanced, with the positive class (fraud) representing only 0.172% of all transactions.

The dataset includes only numerical variables, many of which were transformed using Principal Component Analysis (PCA). Due to confidentiality issues, we cannot provide details about the original features or further background information. The 'Target' feature is the response variable, indicating fraud with a value of 1 and non-fraudulent transactions with a value of 0.

In [ ]:
# importe bibliotecas para detecção de fraude
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# importe bibliotecas para pré-processamento
from sklearn.model_selection import train_test_split

# biblioteca para construção de modelos de machine learning
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
import lightgbm as lgb

# bibliotecas para avaliar modelos de machine learning
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score

# importe bibliotecas para construir dados desbalanceados
from imblearn.over_sampling import SMOTE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/csv/transferencias.csv')
df.head()

,Timestamp,pais,cidade,bairro,cep,ip,dia,hora,minuto,so,...,android,ios,compras,hist_navegacao,relacionamento,ind_seguranca,tempo_transacao,lim_crt,hist_saldo,Target
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [ ]:
# informações sobre dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Timestamp        284807 non-null  float64
 1   pais             284807 non-null  float64
 2   cidade           284807 non-null  float64
 3   bairro           284807 non-null  float64
 4   cep              284807 non-null  float64
 5   ip               284807 non-null  float64
 6   dia              284807 non-null  float64
 7   hora             284807 non-null  float64
 8   minuto           284807 non-null  float64
 9   so               284807 non-null  float64
 10  valor            284807 non-null  float64
 11  antecedentes     284807 non-null  float64
 12  reclamacoes      284807 non-null  float64
 13  qtd_trans        284807 non-null  float64
 14  crt              284807 non-null  float64
 15  limite_global    284807 non-null  float64
 16  tipo_crt         284807 non-null  floa

In [ ]:
# valores nulos
df.isnull().sum()

,0
Timestamp,0
pais,0
cidade,0
bairro,0
cep,0
ip,0
dia,0
hora,0
minuto,0
so,0


In [ ]:
# checar a variavel Target
df['Target'].value_counts()
#sns.countplot(df['Target'])

,count
Target,
0,284315
1,492


In [ ]:
# correlações com o Target
df.corr()['Target'].sort_values(ascending=False)

,Target
Target,1.000000
antecedentes,0.154876
cep,0.133447
cidade,0.091289
android,0.040413
emprestimos,0.034783
navegador,0.020090
minuto,0.019875
tempo_transacao,0.017580
lim_crt,0.009536


In [ ]:
# correlação de spearman
df.corr(method='spearman')

,Timestamp,pais,cidade,bairro,cep,ip,dia,hora,minuto,so,...,android,ios,compras,hist_navegacao,relacionamento,ind_seguranca,tempo_transacao,lim_crt,hist_saldo,Target
Timestamp,1.000000,0.190647,-0.006419,-0.473348,-0.142156,0.226004,-0.111408,0.116722,-0.103882,0.017986,...,0.096564,0.132651,0.148505,-0.015980,-0.259220,0.000132,-0.035063,-0.143200,-0.040007,-0.011692
pais,0.190647,1.000000,-0.384328,-0.412095,0.101011,-0.106632,-0.086561,-0.227650,-0.236140,0.138614,...,-0.061958,-0.028827,0.145440,-0.000315,0.049291,0.006380,-0.104815,-0.207967,-0.086663,-0.042424
cidade,-0.006419,-0.384328,1.000000,0.073967,0.155042,0.342701,-0.110393,0.426320,0.006980,-0.077880,...,-0.098255,-0.056478,-0.094982,0.012617,-0.085682,0.033865,0.129234,0.137976,-0.502052,0.051062
bairro,-0.473348,-0.412095,0.073967,1.000000,0.060812,-0.223849,0.171263,-0.074582,0.164374,-0.030876,...,-0.040234,0.004404,-0.140076,-0.000559,0.031854,-0.016689,0.140724,0.180169,-0.000236,-0.059278
cep,-0.142156,0.101011,0.155042,0.060812,1.000000,0.021367,0.075185,0.059920,-0.009608,0.105248,...,0.038960,-0.004931,0.009615,-0.008782,0.060647,-0.064357,-0.033634,0.004757,-0.022117,0.063045
ip,0.226004,-0.106632,0.342701,-0.223849,0.021367,1.000000,-0.007577,0.431573,-0.057022,-0.056580,...,-0.040364,-0.010920,-0.108300,-0.023076,-0.015618,0.029513,0.012883,-0.027806,-0.314061,-0.030147
dia,-0.111408,-0.086561,-0.110393,0.171263,0.075185,-0.007577,1.000000,-0.292680,0.444053,0.028864,...,0.034595,0.040370,-0.050293,-0.135166,-0.011131,-0.023932,0.073772,0.020459,0.205452,-0.038410
hora,0.116722,-0.227650,0.426320,-0.074582,0.059920,0.431573,-0.292680,1.000000,-0.389213,-0.100252,...,-0.022250,-0.023750,-0.186343,-0.012879,-0.046468,0.016724,-0.094494,0.056405,-0.030003,-0.048308
minuto,-0.103882,-0.236140,0.006980,0.164374,-0.009608,-0.057022,0.444053,-0.389213,1.000000,-0.002511,...,0.076569,0.019775,0.075212,-0.024536,-0.074303,-0.007648,0.189916,0.089700,0.004371,0.022706
so,0.017986,0.138614,-0.077880,-0.030876,0.105248,-0.056580,0.028864,-0.100252,-0.002511,1.000000,...,-0.065773,0.001958,0.074164,-0.001572,-0.022575,0.026677,-0.001804,-0.103743,-0.077437,-0.049499


In [ ]:
# analise descritiva
df.describe()

,Timestamp,pais,cidade,bairro,cep,ip,dia,hora,minuto,so,...,android,ios,compras,hist_navegacao,relacionamento,ind_seguranca,tempo_transacao,lim_crt,hist_saldo,Target
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.168375e-15,3.416908e-16,-1.379537e-15,2.074095e-15,9.604066e-16,1.487313e-15,-5.556467e-16,1.213481e-16,-2.406331e-15,...,1.654067e-16,-3.568593e-16,2.578648e-16,4.473266e-15,5.340915e-16,1.683437e-15,-3.660091e-16,-1.227390e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


# Data Pre-Processing

* Is there class (target) imbalance?
* It can also be tricky to deal with outliers if we don't understand the problem domain and context well enough.



In [ ]:
# separar dados para treino e teste
X = df.drop(columns=['Target'], axis=1)
y = df['Target']

In [ ]:
# balanceamento com smote
# criar registros da classe positiva (1)
smt = SMOTE(random_state=42)
X, y = smt.fit_resample(X, y)

In [ ]:
# contagem do target após o balanceamento: use bincount
print(np.bincount(y))


[284315 284315]


The stratify=y argument in scikit-learn's train_test_split function is used to ensure that the proportion of the target class (dependent variable) is maintained in the training set and test set. This is especially important when you are dealing with imbalanced data, that is, when the classes of the target variable are not represented equally in the data set.

In [ ]:
# separar dados de treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)


# Creating the Predictive Fraud Detection Machine

#XGBoost - Accuracy:  0.9998124146340034 - F1-score: 1.0
* We created the Predictive Machine and fit the model to the data we prepared for training.

* Then we proceed to classify with the data we have prepared for evaluation.

* Lastly, we evaluate model performance with key classification metrics.

In [ ]:
# construa o modelo xgboost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)
xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
# passando os dados de teste para o modelo xgb_model
y_pred_xgb = xgb_model.predict(X_val)
y_pred_xgb

array([1, 0, 0, ..., 1, 0, 0])

## EVALUATING XGBOOST

In [ ]:
# comparando os dados de validação com os dados de predição
validação = pd.DataFrame({'y_val': y_val, 'y_pred_xgb': y_pred_xgb})
validação

,y_val,y_pred_xgb
363369,1,1
158118,0,0
202455,0,0
115521,0,0
193355,0,0
...,...,...
185330,0,0
84822,0,0
447075,1,1
123340,0,0


In [ ]:
# avalie o modelo com métricas, acurácia e matrix de confusão
print(classification_report(y_val, y_pred_xgb))
print('Acurácia: ', accuracy_score(y_val, y_pred_xgb))
#confusion matrix
print(confusion_matrix(y_val, y_pred_xgb))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       1.00      1.00      1.00     85294

    accuracy                           1.00    170589
   macro avg       1.00      1.00      1.00    170589
weighted avg       1.00      1.00      1.00    170589

Acurácia:  0.9998124146340034
[[85264    31]
 [    1 85293]]


#LightGBM - Accuracy:  0.9849521364214574 - f1-score: 0.99


In [ ]:
# crie os dados de treino para o lightgbm
lgb_train = lgb.Dataset(X_train, y_train)

In [ ]:
# setting parameter for lightgbm
params = {'num_leaves':1000, 'objective':'binary', 'max_depth':7, 'learning_rate':.01, 'max_bin':200}
params['metric'] = ['auc', 'binary_logloss']

In [ ]:
# treinar o modelo com lightgbm
lgb_model = lgb.train(params, lgb_train, num_boost_round=50)


[LightGBM] [Info] Number of positive: 199021, number of negative: 199020
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.117686 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6000
[LightGBM] [Info] Number of data points in the train set: 398041, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500001 -> initscore=0.000005
[LightGBM] [Info] Start training from score 0.000005
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive ga

In [ ]:
# Passando dados de validação para o modelo
y_pred_lgb = lgb_model.predict(X_val)

## EVALUATING LGBM

In [ ]:
# comparando os dados de validação com os dados de predição
validação = pd.DataFrame({'y_val': y_val, 'y_pred_lgb': y_pred_lgb})
validação

,y_val,y_pred_lgb
363369,1,0.696905
158118,0,0.304971
202455,0,0.337166
115521,0,0.329696
193355,0,0.313699
...,...,...
185330,0,0.304971
84822,0,0.304971
447075,1,0.696905
123340,0,0.304971


In [ ]:
y_pred_lgb.size

170589

In [ ]:
# converter probabilidades em 0 ou 1
for i in range(0, 170589):
    if y_pred_lgb[i] >= 0.5:   # set threshold to 0.5
        y_pred_lgb[i] = 1
    else:
      y_pred_lgb[i]=0

In [ ]:
# comparando os dados de validação com os dados de predição
validação_probabilidade = pd.DataFrame({'y_val': y_val, 'y_pred_lgb': y_pred_lgb})
validação_probabilidade

,y_val,y_pred_lgb
363369,1,1.0
158118,0,0.0
202455,0,0.0
115521,0,0.0
193355,0,0.0
...,...,...
185330,0,0.0
84822,0,0.0
447075,1,1.0
123340,0,0.0


In [ ]:
# avalie o modelo com métricas, acurácia e matrix de confusão
print(classification_report(y_val, y_pred_lgb))
print('Acurácia: ', accuracy_score(y_val, y_pred_lgb))
#confusion matrix
print(confusion_matrix(y_val, y_pred_lgb))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99     85295
           1       0.99      0.98      0.98     85294

    accuracy                           0.98    170589
   macro avg       0.99      0.98      0.98    170589
weighted avg       0.99      0.98      0.98    170589

Acurácia:  0.9849521364214574
[[84591   704]
 [ 1863 83431]]


# Random Forest - 0.9998300007620655 - f1-score: 1.0

In [ ]:
# construindo modelo com randomforest
rf_model = RandomForestClassifier(random_state=42)
rf_model = rf_model.fit(X_train, y_train)
rf_model

RandomForestClassifier(random_state=42)

In [ ]:
# passando dados de validação para a predição
y_pred_rf = rf_model.predict(X_val)

## EVALUATING RANDOM FOREST

In [ ]:
# comparando os dados de validação com os dados de predição
validação = pd.DataFrame({'y_val': y_val, 'y_pred_rf': y_pred_rf})
validação

,y_val,y_pred_rf
363369,1,1
158118,0,0
202455,0,0
115521,0,0
193355,0,0
...,...,...
185330,0,0
84822,0,0
447075,1,1
123340,0,0


In [ ]:
# avalie o modelo randomforest quanto a metricas, acuracia e confusion matrix
print(classification_report(y_val, y_pred_rf))
print('Acurácia: ', accuracy_score(y_val, y_pred_rf))
#confusion matrix
print(confusion_matrix(y_val, y_pred_rf))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       1.00      1.00      1.00     85294

    accuracy                           1.00    170589
   macro avg       1.00      1.00      1.00    170589
weighted avg       1.00      1.00      1.00    170589

Acurácia:  0.9998300007620655
[[85268    27]
 [    2 85292]]


# Saving the model

In [ ]:
# Salvando o modelo
import joblib
joblib.dump(rf_model, 'modelo_rf.pkl')

['modelo_rf.pkl']

In [ ]:
# usando o modelo pkl
modelo_carregado = joblib.load('modelo_rf.pkl')

In [ ]:
modelo_carregado.predict(X_test)

array([1, 0, 0, ..., 1, 0, 0])